Cell 1: Imports en Gemeenschappelijke Functies

In [13]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Functie om minuten per bar te krijgen op basis van timeframe
def get_minutes_per_bar(timeframe):
    timeframe_minutes = {
        mt5.TIMEFRAME_M1: 1,
        mt5.TIMEFRAME_M5: 5,
        mt5.TIMEFRAME_M15: 15,
        mt5.TIMEFRAME_M30: 30,
        mt5.TIMEFRAME_H1: 60,
        mt5.TIMEFRAME_H4: 240,
        mt5.TIMEFRAME_D1: 1440,
    }
    return timeframe_minutes.get(timeframe, None)

# Functie om timeframe om te zetten naar een string
def get_timeframe_str(timeframe):
    timeframe_str = {
        mt5.TIMEFRAME_M1: 'M1',
        mt5.TIMEFRAME_M5: 'M5',
        mt5.TIMEFRAME_M15: 'M15',
        mt5.TIMEFRAME_M30: 'M30',
        mt5.TIMEFRAME_H1: 'H1',
        mt5.TIMEFRAME_H4: 'H4',
        mt5.TIMEFRAME_D1: 'D1',
    }
    return timeframe_str.get(timeframe, 'Unknown')

# Vertalingsfuncties voor MT5 modi
def translate_trade_mode(trade_mode):
    if trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        return "Disabled"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_LONGONLY:
        return "Long Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_SHORTONLY:
        return "Short Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_CLOSEONLY:
        return "Close Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_FULL:
        return "Full"
    return "Unknown"

def translate_filling_mode(filling_mode):
    modes = []
    if filling_mode & mt5.SYMBOL_FILLING_FOK:
        modes.append("Fill or Kill")
    if filling_mode & mt5.SYMBOL_FILLING_IOC:
        modes.append("Immediate or Cancel")
    if filling_mode & mt5.SYMBOL_FILLING_RETURN:
        modes.append("Return")
    return ", ".join(modes) if modes else "None"

def translate_expiration_mode(expiration_mode):
    modes = []
    if expiration_mode & mt5.SYMBOL_EXPIRATION_GTC:
        modes.append("Good Till Cancelled")
    if expiration_mode & mt5.SYMBOL_EXPIRATION_DAY:
        modes.append("Day")
    if expiration_mode & mt5.SYMBOL_EXPIRATION_SPECIFIED:
        modes.append("Specified")
    if expiration_mode & mt5.SYMBOL_EXPIRATION_SPECIFIED_DAY:
        modes.append("Specified Day")
    return ", ".join(modes) if modes else "None"

def translate_order_mode(order_mode):
    modes = []
    if order_mode & mt5.SYMBOL_ORDER_MARKET:
        modes.append("Market")
    if order_mode & mt5.SYMBOL_ORDER_LIMIT:
        modes.append("Limit")
    if order_mode & mt5.SYMBOL_ORDER_STOP:
        modes.append("Stop")
    if order_mode & mt5.SYMBOL_ORDER_STOP_LIMIT:
        modes.append("Stop Limit")
    if order_mode & mt5.SYMBOL_ORDER_SL:
        modes.append("Stop Loss")
    if order_mode & mt5.SYMBOL_ORDER_TP:
        modes.append("Take Profit")
    if order_mode & mt5.SYMBOL_ORDER_CLOSURE:
        modes.append("Closure")
    return ", ".join(modes) if modes else "None"

Cell 2: Verbinding met MT5

In [14]:
def connect_to_mt5(path=None):
    """Maakt verbinding met MT5 terminal met foutafhandeling"""
    if not mt5.initialize(path=path):
        print(f"Initialize() mislukt, error code = {mt5.last_error()}")
        return False

    if not mt5.terminal_info():
        print(f"Kon geen verbinding maken met terminal, error code = {mt5.last_error()}")
        return False

    account_info = mt5.account_info()
    if account_info:
        print(f"Verbonden met account: {account_info.login}")
        print(f"Server: {account_info.server}")
        print(f"Balance: {account_info.balance}")
        print(f"Leverage: 1:{account_info.leverage}")
        return True
    return False

# Initialiseer verbinding (pas het pad aan naar jouw MT5 terminal)
mt5_connected = connect_to_mt5(path="C:\\Program Files\\FTMO Global Markets MT5 Terminal\\terminal64.exe")

Verbonden met account: 1520533067
Server: FTMO-Demo2
Balance: 79997.56
Leverage: 1:100


Cell 3: Liquiditeitsanalyse

In [15]:
def analyze_liquidity(symbols, period=mt5.TIMEFRAME_H1, lookback=100):
    """
    Analyseert liquiditeit gebaseerd op tick volume

    Parameters:
    - symbols: Lijst met te analyseren symbolen
    - period: Timeframe voor analyse
    - lookback: Aantal candles om terug te kijken
    """
    to_date = datetime.now()
    minutes_per_bar = get_minutes_per_bar(period)
    if minutes_per_bar is None:
        raise ValueError(f"Unsupported timeframe: {period}")
    from_date = to_date - timedelta(minutes=lookback * minutes_per_bar)

    results = []
    for symbol in symbols:
        rates = mt5.copy_rates_range(symbol, period, from_date, to_date)
        if rates is None or len(rates) == 0:
            continue

        df = pd.DataFrame(rates)
        avg_volume = df['tick_volume'].mean()
        max_volume = df['tick_volume'].max()
        min_volume = df['tick_volume'].min()
        df['hl_range'] = df['high'] - df['low']
        avg_range = df['hl_range'].mean()

        symbol_info = mt5.symbol_info(symbol)
        result = {
            'symbol': symbol,
            'avg_tick_volume': int(avg_volume),
            'max_tick_volume': int(max_volume),
            'min_tick_volume': int(min_volume),
            'avg_hl_range': float(avg_range),
            'timeframe': get_timeframe_str(period),
            'spread': symbol_info.spread if symbol_info else None,
            'trade_mode': translate_trade_mode(symbol_info.trade_mode) if symbol_info else None
        }
        results.append(result)

    return pd.DataFrame(results)

# Definieer symbolen om te analyseren
test_symbols = ["EURUSD", "GBPUSD", "USDJPY"]

if mt5_connected:
    liquidity_df = analyze_liquidity(test_symbols)
    print("\nLiquiditeitsanalyse:")
    liquidity_df_sorted = liquidity_df.sort_values('avg_tick_volume', ascending=False)
    print(liquidity_df_sorted[['symbol', 'avg_tick_volume', 'avg_hl_range', 'spread', 'trade_mode']].head(10))
else:
    print("MT5 niet verbonden, kan liquiditeitsanalyse niet uitvoeren.")



Liquiditeitsanalyse:
   symbol  avg_tick_volume  avg_hl_range  spread trade_mode
2  USDJPY             3367      0.260123      17       Full
1  GBPUSD             2456      0.001519      15       Full
0  EURUSD             2213      0.001545       9       Full


Cell 4: Gedetailleerde Symboolinformatie

In [16]:
import MetaTrader5 as mt5

# Definieer MT5 constanten die niet direct beschikbaar zijn in de Python API
SYMBOL_FILLING_FOK = 1
SYMBOL_FILLING_IOC = 2
SYMBOL_FILLING_RETURN = 4
SYMBOL_EXPIRATION_GTC = 1
SYMBOL_EXPIRATION_DAY = 2
SYMBOL_EXPIRATION_SPECIFIED = 4
SYMBOL_EXPIRATION_SPECIFIED_DAY = 8
SYMBOL_ORDER_MARKET = 1
SYMBOL_ORDER_LIMIT = 2
SYMBOL_ORDER_STOP = 4
SYMBOL_ORDER_STOP_LIMIT = 8
SYMBOL_ORDER_SL = 16
SYMBOL_ORDER_TP = 32
SYMBOL_ORDER_CLOSURE = 64

# Vertalingsfuncties
def translate_filling_mode(filling_mode):
    """Vertaalt filling mode naar leesbare tekst"""
    modes = []
    if filling_mode & SYMBOL_FILLING_FOK:
        modes.append("Fill or Kill")
    if filling_mode & SYMBOL_FILLING_IOC:
        modes.append("Immediate or Cancel")
    if filling_mode & SYMBOL_FILLING_RETURN:
        modes.append("Return")
    return ", ".join(modes) if modes else "None"

def translate_expiration_mode(expiration_mode):
    """Vertaalt expiration mode naar leesbare tekst"""
    modes = []
    if expiration_mode & SYMBOL_EXPIRATION_GTC:
        modes.append("Good Till Cancelled")
    if expiration_mode & SYMBOL_EXPIRATION_DAY:
        modes.append("Day")
    if expiration_mode & SYMBOL_EXPIRATION_SPECIFIED:
        modes.append("Specified")
    if expiration_mode & SYMBOL_EXPIRATION_SPECIFIED_DAY:
        modes.append("Specified Day")
    return ", ".join(modes) if modes else "None"

def translate_order_mode(order_mode):
    """Vertaalt order mode naar leesbare tekst"""
    modes = []
    if order_mode & SYMBOL_ORDER_MARKET:
        modes.append("Market")
    if order_mode & SYMBOL_ORDER_LIMIT:
        modes.append("Limit")
    if order_mode & SYMBOL_ORDER_STOP:
        modes.append("Stop")
    if order_mode & SYMBOL_ORDER_STOP_LIMIT:
        modes.append("Stop Limit")
    if order_mode & SYMBOL_ORDER_SL:
        modes.append("Stop Loss")
    if order_mode & SYMBOL_ORDER_TP:
        modes.append("Take Profit")
    if order_mode & SYMBOL_ORDER_CLOSURE:
        modes.append("Closure")
    return ", ".join(modes) if modes else "None"

def translate_trade_mode(trade_mode):
    """Vertaalt trade mode naar leesbare tekst"""
    if trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        return "Disabled"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_LONGONLY:
        return "Long Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_SHORTONLY:
        return "Short Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_CLOSEONLY:
        return "Close Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_FULL:
        return "Full"
    return "Unknown"

def translate_trade_calc_mode(mode):
    """Vertaalt trade_calc_mode naar leesbare tekst"""
    modes = {
        0: "Forex",
        1: "CFD",
        2: "Futures",
        3: "Exchange Futures",
        4: "Exchange Stocks",
    }
    return modes.get(mode, "Onbekend")

def get_detailed_symbol_info(symbol):
    """Haalt gedetailleerde informatie op over een specifiek symbool"""
    symbol_info = mt5.symbol_info(symbol)
    if not symbol_info:
        print(f"Symbol '{symbol}' not found.")
        return None

    tick = mt5.symbol_info_tick(symbol)
    details = {
        "Basis Info": {
            "Naam": symbol_info.name,
            "Beschrijving": symbol_info.description,
            "ISIN": symbol_info.isin,
            "Basis Valuta": symbol_info.currency_base,
            "Winst Valuta": symbol_info.currency_profit,
            "Categorie": symbol_info.path,
        },
        "Handelscondities": {
            "Contract Grootte": symbol_info.trade_contract_size,
            "Minimum Lot": symbol_info.volume_min,
            "Maximum Lot": symbol_info.volume_max,
            "Lot Stap": symbol_info.volume_step,
            "Swap Long": symbol_info.swap_long,
            "Swap Short": symbol_info.swap_short,
            "Swap Rollover 3-dagen": symbol_info.swap_rollover3days,
            "Margin Initieel": symbol_info.margin_initial,
            "Margin Onderhoud": symbol_info.margin_maintenance,
            "Hedge Margin": symbol_info.margin_hedged,
            "Margin Berekening Modus": translate_trade_calc_mode(symbol_info.trade_calc_mode),
        },
        "Prijsinformatie": {
            "Bid": tick.bid if tick else None,
            "Ask": tick.ask if tick else None,
            "Spread": (tick.ask - tick.bid) if tick else None,
            "Spread in punten": symbol_info.spread,
            "Spread is Variabel": bool(symbol_info.spread_float),
            "Stops Level": symbol_info.trade_stops_level,
            "Freeze Level": symbol_info.trade_freeze_level,
        },
        "Order Condities": {
            "Vul Modes": translate_filling_mode(symbol_info.filling_mode),
            "Expiratie Modes": translate_expiration_mode(symbol_info.expiration_mode),
            "Order Modes": translate_order_mode(symbol_info.order_mode),
            "Trade Modes": translate_trade_mode(symbol_info.trade_mode),
        }
    }

    print("\nGedetailleerde informatie:")
    for category, info in details.items():
        print(f"\n{category}:")
        for key, value in info.items():
            print(f"  {key}: {value}")
    return details

# Voorbeeldgebruik
mt5_connected = mt5.initialize()  # Initialisatie voor testdoeleinden
if mt5_connected:
    example_symbol = "EURUSD"
    get_detailed_symbol_info(example_symbol)
else:
    print("MT5 niet verbonden, kan gedetailleerde symboolinformatie niet ophalen.")


Gedetailleerde informatie:

Basis Info:
  Naam: EURUSD
  Beschrijving: Euro vs United States Dollar
  ISIN: 
  Basis Valuta: EUR
  Winst Valuta: USD
  Categorie: Forex\EURUSD

Handelscondities:
  Contract Grootte: 100000.0
  Minimum Lot: 0.01
  Maximum Lot: 50.0
  Lot Stap: 0.01
  Swap Long: -10.92
  Swap Short: 1.68
  Swap Rollover 3-dagen: 3
  Margin Initieel: 0.0
  Margin Onderhoud: 0.0
  Hedge Margin: 200000.0
  Margin Berekening Modus: Forex

Prijsinformatie:
  Bid: 1.08127
  Ask: 1.08136
  Spread: 9.000000000014552e-05
  Spread in punten: 9
  Spread is Variabel: True
  Stops Level: 0
  Freeze Level: 0

Order Condities:
  Vul Modes: Fill or Kill, Immediate or Cancel
  Expiratie Modes: Good Till Cancelled, Day, Specified, Specified Day
  Order Modes: Market, Limit, Stop, Stop Limit, Stop Loss, Take Profit, Closure
  Trade Modes: Full


Mark tijden visualisatie

In [19]:
import MetaTrader5 as mt5
import datetime
from dateutil import tz
import pytz

# Constanten voor MT5 (zoals eerder gedefinieerd)
SYMBOL_FILLING_FOK = 1
SYMBOL_FILLING_IOC = 2
SYMBOL_FILLING_RETURN = 4
SYMBOL_EXPIRATION_GTC = 1
SYMBOL_EXPIRATION_DAY = 2
SYMBOL_EXPIRATION_SPECIFIED = 4
SYMBOL_EXPIRATION_SPECIFIED_DAY = 8
SYMBOL_ORDER_MARKET = 1
SYMBOL_ORDER_LIMIT = 2
SYMBOL_ORDER_STOP = 4
SYMBOL_ORDER_STOP_LIMIT = 8
SYMBOL_ORDER_SL = 16
SYMBOL_ORDER_TP = 32
SYMBOL_ORDER_CLOSURE = 64

# Vertalingsfuncties (ongewijzigd)
def translate_filling_mode(filling_mode):
    """Vertaalt filling mode naar leesbare tekst"""
    modes = []
    if filling_mode & SYMBOL_FILLING_FOK:
        modes.append("Fill or Kill")
    if filling_mode & SYMBOL_FILLING_IOC:
        modes.append("Immediate or Cancel")
    if filling_mode & SYMBOL_FILLING_RETURN:
        modes.append("Return")
    return ", ".join(modes) if modes else "None"

def translate_expiration_mode(expiration_mode):
    """Vertaalt expiration mode naar leesbare tekst"""
    modes = []
    if expiration_mode & SYMBOL_EXPIRATION_GTC:
        modes.append("Good Till Cancelled")
    if expiration_mode & SYMBOL_EXPIRATION_DAY:
        modes.append("Day")
    if expiration_mode & SYMBOL_EXPIRATION_SPECIFIED:
        modes.append("Specified")
    if expiration_mode & SYMBOL_EXPIRATION_SPECIFIED_DAY:
        modes.append("Specified Day")
    return ", ".join(modes) if modes else "None"

def translate_order_mode(order_mode):
    """Vertaalt order mode naar leesbare tekst"""
    modes = []
    if order_mode & SYMBOL_ORDER_MARKET:
        modes.append("Market")
    if order_mode & SYMBOL_ORDER_LIMIT:
        modes.append("Limit")
    if order_mode & SYMBOL_ORDER_STOP:
        modes.append("Stop")
    if order_mode & SYMBOL_ORDER_STOP_LIMIT:
        modes.append("Stop Limit")
    if order_mode & SYMBOL_ORDER_SL:
        modes.append("Stop Loss")
    if order_mode & SYMBOL_ORDER_TP:
        modes.append("Take Profit")
    if order_mode & SYMBOL_ORDER_CLOSURE:
        modes.append("Closure")
    return ", ".join(modes) if modes else "None"

def translate_trade_mode(trade_mode):
    """Vertaalt trade mode naar leesbare tekst"""
    if trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        return "Disabled"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_LONGONLY:
        return "Long Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_SHORTONLY:
        return "Short Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_CLOSEONLY:
        return "Close Only"
    elif trade_mode == mt5.SYMBOL_TRADE_MODE_FULL:
        return "Full"
    return "Unknown"

def translate_trade_calc_mode(mode):
    """Vertaalt trade_calc_mode naar leesbare tekst"""
    modes = {
        0: "Forex",
        1: "CFD",
        2: "Futures",
        3: "Exchange Futures",
        4: "Exchange Stocks",
    }
    return modes.get(mode, "Onbekend")

def get_platform_time_and_trading_hours(symbols=None):
    """Haalt de MT5-platformtijd op en vergelijkt deze met lokale tijd, plus handelstijden van symbolen"""
    if not mt5.initialize():
        print("MT5 initialisatie mislukt")
        return None, None, None

    # Haal de server(platform)tijd op via de laatste tick van een symbool (EURUSD als voorbeeld)
    tick = mt5.symbol_info_tick("EURUSD")
    if not tick:
        print("Kon server-tijd niet ophalen via tick data.")
        server_time_dt = datetime.datetime.now(pytz.UTC)  # Fallback naar UTC
    else:
        server_time_dt = datetime.datetime.fromtimestamp(tick.time, tz=pytz.UTC)

    # Haal lokale Windows-tijd op
    local_time = datetime.datetime.now(tz.tzlocal())

    # Bereken tijdsverschil
    time_difference = server_time_dt - local_time

    print("\nTijdsinformatie:")
    print(f"MT5 Server Tijd: {server_time_dt} (UTC)")
    print(f"Locale Windows Tijd: {local_time}")
    print(f"Tijdsverschil (Server - Lokaal): {time_difference}")

    # Belangrijke symbolen (aanpasbaar)
    major_symbols = symbols if symbols else ["EURUSD", "GBPUSD", "USDJPY", "AUDUSD", "USDCAD"]

    # Handelstijden ophalen (indicatief)
    trading_hours = {}
    for symbol in major_symbols:
        symbol_info = mt5.symbol_info(symbol)
        if not symbol_info:
            print(f"Symbol '{symbol}' niet gevonden.")
            continue

        trade_mode = translate_trade_mode(symbol_info.trade_mode)
        if trade_mode == "Full":
            # Voor Forex: indicatieve tijden (24/5, zondag 22:00 UTC - vrijdag 22:00 UTC)
            trading_hours[symbol] = {
                "Maandag": "00:00 - 23:59",
                "Dinsdag": "00:00 - 23:59",
                "Woensdag": "00:00 - 23:59",
                "Donderdag": "00:00 - 23:59",
                "Vrijdag": "00:00 - 23:59",
                "Zaterdag": "Gesloten",
                "Zondag": "Gesloten (opent 22:00 UTC)"
            }
        elif trade_mode in ["Disabled", "Close Only"]:
            trading_hours[symbol] = {"Alle dagen": "Handel uitgeschakeld of alleen sluiten"}
        else:
            trading_hours[symbol] = {"Info": f"Specifieke modus: {trade_mode}, raadpleeg broker"}

    print("\nHandelstijden van belangrijke symbolen (in server tijdzone, UTC):")
    for symbol, hours in trading_hours.items():
        print(f"\n{symbol}:")
        for day, time in hours.items():
            print(f"  {day}: {time}")

    return server_time_dt, local_time, trading_hours

# Voorbeeldgebruik
if __name__ == "__main__":
    mt5_connected = mt5.initialize()
    if mt5_connected:
        server_time, local_time, trading_hours = get_platform_time_and_trading_hours(None)
        mt5.shutdown()
    else:
        print("MT5 niet verbonden, kan informatie niet ophalen.")


Tijdsinformatie:
MT5 Server Tijd: 2025-03-21 23:54:57+00:00 (UTC)
Locale Windows Tijd: 2025-03-23 11:55:04.936608+01:00
Tijdsverschil (Server - Lokaal): -2 days, 12:59:52.063392

Handelstijden van belangrijke symbolen (in server tijdzone, UTC):

EURUSD:
  Maandag: 00:00 - 23:59
  Dinsdag: 00:00 - 23:59
  Woensdag: 00:00 - 23:59
  Donderdag: 00:00 - 23:59
  Vrijdag: 00:00 - 23:59
  Zaterdag: Gesloten
  Zondag: Gesloten (opent 22:00 UTC)

GBPUSD:
  Maandag: 00:00 - 23:59
  Dinsdag: 00:00 - 23:59
  Woensdag: 00:00 - 23:59
  Donderdag: 00:00 - 23:59
  Vrijdag: 00:00 - 23:59
  Zaterdag: Gesloten
  Zondag: Gesloten (opent 22:00 UTC)

USDJPY:
  Maandag: 00:00 - 23:59
  Dinsdag: 00:00 - 23:59
  Woensdag: 00:00 - 23:59
  Donderdag: 00:00 - 23:59
  Vrijdag: 00:00 - 23:59
  Zaterdag: Gesloten
  Zondag: Gesloten (opent 22:00 UTC)

AUDUSD:
  Maandag: 00:00 - 23:59
  Dinsdag: 00:00 - 23:59
  Woensdag: 00:00 - 23:59
  Donderdag: 00:00 - 23:59
  Vrijdag: 00:00 - 23:59
  Zaterdag: Gesloten
  Zondag: G